##  Lecture du fichier CSV des discours américains

In [1]:
import pandas as pd

df = pd.read_csv("discours_US.csv", sep="\t")

##  Analyse de la répartition des orateurs (speakers)

In [2]:
df['speaker'].value_counts()


speaker
CLINTON    93
TRUMP      71
Name: count, dtype: int64

## Création du corpus à partir des discours

In [3]:
import sys
sys.path.append("../src")

from Corpus import Corpus

corpus = Corpus("discours_US")

for i, row in df.iterrows():
    corpus.add_document(
        titre=row["descr"],        # description du discours
        auteur=row["speaker"],     # orateur
        date=row["date"],          # déjà bon format
        url=row["link"],           # lien vers l'archive
        texte=row["text"]          # contenu du discours
    )



##  Vérification du corpus construit

In [4]:
corpus.ndoc, corpus.naut


(164, 2)

##  Recherche simple par mot-clé dans le corpus

In [5]:
corpus.search("america")


[('doc_0', <document.Document at 0x1746700dd90>),
 ('doc_2', <document.Document at 0x17467586a10>),
 ('doc_3', <document.Document at 0x17468ca2d90>),
 ('doc_4', <document.Document at 0x17468ca34d0>),
 ('doc_5', <document.Document at 0x17468ca2c10>),
 ('doc_6', <document.Document at 0x17468ca35d0>),
 ('doc_7', <document.Document at 0x17468ca3110>),
 ('doc_8', <document.Document at 0x17468ca3550>),
 ('doc_9', <document.Document at 0x17468ca3610>),
 ('doc_10', <document.Document at 0x17468ca3690>),
 ('doc_11', <document.Document at 0x17468ca3710>),
 ('doc_12', <document.Document at 0x17468ca3790>),
 ('doc_13', <document.Document at 0x17468ca3810>),
 ('doc_14', <document.Document at 0x17468ca1b50>),
 ('doc_15', <document.Document at 0x17468ca3910>),
 ('doc_16', <document.Document at 0x17468ca3990>),
 ('doc_17', <document.Document at 0x17468ca3a10>),
 ('doc_19', <document.Document at 0x17468ca3bd0>),
 ('doc_20', <document.Document at 0x17468ca3c90>),
 ('doc_23', <document.Document at 0x1746

## 🔎  Analyse de contexte avec le concordancier

In [6]:
corpus.concorde("america", window=30)[:10]


[('use when families are strong,',
  'America',
  "is strong. So I'm hitting the",
  'doc_0'),
 ('esses starting and growing in',
  'America',
  'again. We have stalled out. I',
  'doc_2'),
 ('ths about race and justice in',
  'America',
  '. There is something profoundl',
  'doc_3'),
 ('ne this shared vision of what',
  'America',
  'can be and should be. I learn',
  'doc_3'),
 ('sweeping small-town and rural',
  'America',
  'as well. We have to do more a',
  'doc_3'),
 ('sty about race and justice in',
  'America',
  '. And, yes, a time for reform.',
  'doc_3'),
 ('use when families are strong,',
  'America',
  'is strong. And I am convinced',
  'doc_4'),
 ('lly undermining what has made',
  'America',
  'unique—the way we have assimi',
  'doc_4'),
 ('iew the right thing to do for',
  'America',
  '. If you compare us to other c',
  'doc_4'),
 ('the kindergarten teachers in',
  'America',
  '. And when you think about val',
  'doc_5')]

## 🤖 Chargement ou construction du moteur TF-IDF (SAFE MODE)

In [7]:
import os
import pickle

from SearchEngine import SearchEngine


ENGINE_PATH = "engine.pkl"

if os.path.exists(ENGINE_PATH):
    print("🔁 Moteur TF-IDF déjà existant. Chargement depuis engine.pkl...")
    
    with open(ENGINE_PATH, "rb") as f:
        engine = pickle.load(f)

else:
    print("⚙️ Aucun moteur trouvé. Construction du moteur TF-IDF...")
    
    engine = SearchEngine(corpus, mode="tfidf")

    with open(ENGINE_PATH, "wb") as f:
        pickle.dump(engine, f)

    print("✅ Moteur TF-IDF construit et sauvegardé.")


🔁 Moteur TF-IDF déjà existant. Chargement depuis engine.pkl...


##  Vérification des dimensions de la matrice TF-IDF

In [8]:
engine.matrix.shape


(164, 12203)

## Recherche multi-mots avec le moteur TF-IDF

In [9]:
engine.search("america freedom", top=5)


100%|██████████| 164/164 [00:00<00:00, 1720.30it/s]


,id,titre,score
90,doc_90,Remarks at a Rally at the James L. Knight Cent...,0.068383
72,doc_72,"Remarks at the XFinity Arena in Everett, Washi...",0.066183
54,doc_54,"Remarks in San Diego, California",0.039486
140,doc_140,Remarks at the Collier County Fairgrounds in N...,0.039355
71,doc_71,Remarks at the Mississippi Coliseum in Jackson...,0.038743
